In [1]:
import os
import pandas as pd

# Attempting to process data types first to get correct format
# Nested the target CSV files into another "resources"
folder_path = 'resources'

# List to store processed data
preprocessed_data = []

# Grab each CSV and read in, manually edit format and data type by column
for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv'):
        
        df = pd.read_csv(os.path.join(folder_path, file_name))

        # Convert percentage columns to decimal numbers
        percentage_columns = ['18-24<hs_grad', '18-24_hs_grad/equiv', '18-24_some_college/associate',
                              '18-24_bachelor/higher', '25+<9th_grade', '25+_9th-12th_grade_nongrad',
                              '25+_hs_grad/equiv', '25+_some_college', "25+_associate's", "25+_bachelor's",
                              '25+>bachelor', '25-34_hs_grad/higher', '25-34_bachelor/higher',
                              '35-44_hs_grad/higher', '35-44_bachelor/higher', '45-64_hs_grad/higher',
                              '45-64_bachelor/higher', '65+_hs_grad/higher', '65+_bachelor/higher',
                              '25+<hs_grad', '25+_hs_grad/equiv.1', '25+_some_college/associate',
                              "25+_bachelor's.1", '25+>bachelor.1']

        for col in percentage_columns:
            df[col] = df[col].str.rstrip('%').str.replace(',', '').astype(float) / 100

        # Convert columns with commas to numeric types
        # Handles the 'O' data types
        for col in df.columns[2:]:
            if df[col].dtype == 'O':  
                df[col] = df[col].str.replace(',', '').astype(float)

        # Append the preprocessed DataFrame to the list
        preprocessed_data.append(df)

# Check each to see if this method works
for i, df in enumerate(preprocessed_data):
    print(f"DataFrame {i+1}:")
    print(df.head())
    print()


DataFrame 1:
         state  year  18-24_count  18-24<hs_grad  18-24_hs_grad/equiv  \
0     Alabama   2010     488349.0          0.199                0.301   
1      Alaska   2010      75023.0          0.188                0.371   
2     Arizona   2010     634568.0          0.196                0.307   
3    Arkansas   2010     291054.0          0.203                0.325   
4  California   2010    3944934.0          0.173                0.276   

   18-24_some_college/associate  18-24_bachelor/higher   25+_count  \
0                         0.439                  0.060   3161521.0   
1                         0.407                  0.034    450577.0   
2                         0.436                  0.060   4146758.0   
3                         0.415                  0.056   1919422.0   
4                         0.473                  0.078  24097200.0   

   25+<9th_grade  25+_9th-12th_grade_nongrad  ...  45-64_bachelor/higher  \
0          0.062                       0.117  ...  

In [2]:

import sqlite3

# Nested path that contains first batch of CSVs
folder_path = 'resources'

# DB Creation:
db_filename = 'Pverty_DB.db'
conn = sqlite3.connect(db_filename)
cursor = conn.cursor()

# Initial table to handle keys by taking just state names and assigning an ID for referencing 
cursor.execute('''CREATE TABLE IF NOT EXISTS states (
                    id INTEGER PRIMARY KEY,
                    state TEXT UNIQUE
                )''')

# Function to preprocess the CSV files and create tables 
# Using REAL data types to include any % values, floats or ints. These CSVs have a variety of numerics, and 2014 - 2015 has a shift
# in data for certain columns
def process_csv(file_path, table_name):
    
    df = pd.read_csv(file_path)

    # Write schema for current table
    schema = ['id INTEGER PRIMARY KEY', 'state_id INTEGER', 'year INTEGER']
    for col in df.columns[2:]:
        schema.append(f'"{col}" REAL')

    # Create Table
    cursor.execute(f'''CREATE TABLE IF NOT EXISTS {table_name} ({", ".join(schema)}, FOREIGN KEY (state_id) REFERENCES states(id))''')

    # Each instance of a state stored
    # Some states have a space after
    states = df['state'].unique()
    for state in states:
        # Remove whitespace from states
        state = state.strip()
        cursor.execute('''INSERT OR IGNORE INTO states (state) VALUES (?)''', (state,))

    # Insert data into current table and check for mismatch
    for index, row in df.iterrows():
        state_id = cursor.execute('''SELECT id FROM states WHERE state = ?''', (row['state'].strip(),)).fetchone()
        if state_id:
            state_id = state_id[0]  # Check if state_id is not None
            cursor.execute(f'''INSERT INTO {table_name} (state_id, year, '''
                           + ', '.join([f'"{col}"' for col in df.columns[2:]]) +
                           ''') VALUES ('''
                           + ', '.join(['?' for _ in range(len(df.columns))]) +
                           ''')''', [state_id] + row.tolist()[1:])
        else:
            print(f"State ID not found for '{row['state']}'.")

# Loop through each CSV file and grab year for table name
for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv'):
        
        year = file_name.split('_')[2].split('.')[0]

        # Skip 2020 (no data)
        if year == '2020':
            continue

        # Write schema for current table
        table_name = f'data_{year}'
        file_path = os.path.join(folder_path, file_name)

        # Process the CSV file and create table in the database
        process_csv(file_path, table_name)

# Commit initial CSV batch to DB
conn.commit()

# Now, adding the new API data as a table - this file is in directory with notebook, not in previous batch
new_csv_file = 'API_Poverty.csv'  
new_df = pd.read_csv(new_csv_file)

# Create table schema
cursor.execute('''CREATE TABLE IF NOT EXISTS API_Data (
                    id INTEGER PRIMARY KEY,
                    state_id INTEGER,
                    median_h_income INTEGER,
                    poverty_rate REAL,
                    poverty_count INTEGER,
                    year INTEGER,
                    FOREIGN KEY (state_id) REFERENCES states(id)
                )''')

# Same process for key state table
# Note "District of Columbia" not in original batch of CSVs so this is removed, shown in output cell
state_ids = {}
for state in new_df['state'].unique():
    # Remove trailing whitespace from state names
    state = state.strip()
    state_id = cursor.execute('''SELECT id FROM states WHERE state = ?''', (state,)).fetchone()
    if state_id:
        state_id = state_id[0]  # Check if state_id is not None
        state_ids[state] = state_id
    else:
        print(f"State '{state}' not found in 'states' table.")

# Load table
for index, row in new_df.iterrows():
    state_id = state_ids.get(row['state'])
    if state_id is not None:
        cursor.execute('''INSERT INTO API_Data (state_id, median_h_income, poverty_rate, poverty_count, year)
                          VALUES (?, ?, ?, ?, ?)''', (state_id, row['median_h_income'], row['poverty_rate'], row['poverty_count'], row['year']))
    else:
        print(f"State ID not found for '{row['state']}'.")

# Commit & close connection
conn.commit()
conn.close()

print("Database created successfully.")


State 'District of Columbia' not found in 'states' table.
State ID not found for 'District of Columbia'.
State ID not found for 'District of Columbia'.
State ID not found for 'District of Columbia'.
State ID not found for 'District of Columbia'.
State ID not found for 'District of Columbia'.
State ID not found for 'District of Columbia'.
State ID not found for 'District of Columbia'.
State ID not found for 'District of Columbia'.
State ID not found for 'District of Columbia'.
State ID not found for 'District of Columbia'.
State ID not found for 'District of Columbia'.
State ID not found for 'District of Columbia'.
Database created successfully.
